In [15]:
import tensorflow as tf
import pandas as pd
import numpy as np
import xarray as xr
import os

In [21]:
df = {}
file_path = os.path.join(os.getcwd(), "forex_wavelet")
for f in os.listdir(file_path):
    
    df[f] = pd.read_csv(os.path.join(file_path, f))
    df[f]['openTime'] = pd.to_datetime(df[f].Date, unit='ns')
    df[f]['openTime'] = pd.DatetimeIndex(df[f]['openTime'] )
    df[f].index = df[f]['openTime']
    df[f].drop(columns=['openTime', 'Date'])
    df[f] = xr.DataArray(df[f], dims=['openTime', 'feature'])
    
        
chart = xr.Dataset(df).to_array(dim='asset')
print(chart)
print(type(chart))

<xarray.DataArray (asset: 7, openTime: 74551, feature: 7)>
array([[[0.81767, '2015-01-01T22:00:00.000000000Z', ..., 14,
         Timestamp('2015-01-01 22:00:00')],
        [0.8173199999999999, '2015-01-01T22:10:00.000000000Z', ..., 18,
         Timestamp('2015-01-01 22:10:00')],
        ...,
        [0.7207600000000001, '2016-12-30T21:40:00.000000000Z', ..., 11,
         Timestamp('2016-12-30 21:40:00')],
        [0.72012, '2016-12-30T21:50:00.000000000Z', ..., 79,
         Timestamp('2016-12-30 21:50:00')]],

       [[1.5583, '2015-01-01T22:00:00.000000000Z', ..., 19,
         Timestamp('2015-01-01 22:00:00')],
        [1.55836, '2015-01-01T22:10:00.000000000Z', ..., 7,
         Timestamp('2015-01-01 22:10:00')],
        ...,
        [1.23248, '2016-12-30T21:40:00.000000000Z', ..., 493,
         Timestamp('2016-12-30 21:40:00')],
        [1.2341799999999998, '2016-12-30T21:50:00.000000000Z', ..., 541,
         Timestamp('2016-12-30 21:50:00')]],

       ...,

       [[6.15319, '2015-0

In [53]:
close_price = chart.sel(feature=['Close_cA','Close_cD'])
close_price = close_price.T
usd_data = np.ones((close_price.shape[0],1))
in_data = np.hstack((usd_data, close_price))
in_data = in_data[1:] / in_data[:-1]
in_data = in_data / in_data[0]

# print(in_data)
m,n = in_data.shape

import logging

logging.basicConfig(level=logging.INFO)



In [52]:
import tensorflow.keras as keras

model = keras.Sequential([
    keras.layers.Conv2D(5, (3, 3), activation='relu', padding='same', input_shape=(n,1,1)),
    keras.layers.Dense(n, activation='softmax')
])


import tensorflow.keras.backend as K

def loss6(y, b):
    gross_return = K.log(K.sum(y * b))
    return K.mean(gross_return)

model.compile(optimizer='adam',
              loss=loss6,
              metrics=['accuracy'])

# model.fit(in_data[:,:,None,None])
model.predict(in_data[:,:,None,None])

array([[[[0.17144188, 0.12742743, 0.12323787, ..., 0.1483949 ,
          0.11146152, 0.10087132]],

        [[0.19304486, 0.15043993, 0.12082243, ..., 0.13612956,
          0.10532903, 0.08877364]],

        [[0.19304486, 0.15043993, 0.12082243, ..., 0.13612956,
          0.10532903, 0.08877364]],

        ...,

        [[0.19304486, 0.15043993, 0.12082243, ..., 0.13612956,
          0.10532903, 0.08877364]],

        [[0.19304486, 0.15043993, 0.12082243, ..., 0.13612956,
          0.10532903, 0.08877364]],

        [[0.13432583, 0.13003659, 0.12620136, ..., 0.13231358,
          0.11342891, 0.12210174]]],


       [[[0.17142116, 0.12742417, 0.12323822, ..., 0.14839086,
          0.11146563, 0.10088033]],

        [[0.1930535 , 0.15044405, 0.1208229 , ..., 0.13612668,
          0.10532859, 0.08877055]],

        [[0.19306554, 0.15043527, 0.12082214, ..., 0.13613838,
          0.1053255 , 0.08876716]],

        ...,

        [[0.19302265, 0.15043971, 0.12082367, ..., 0.1361234 ,
       